In [1]:
import json
import random
import time
from pathlib import Path
import numpy as np
from collections import defaultdict, Counter
from pprint import pprint
import tqdm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import torch


def read_event_dataset(path):
    with open(path) as f:
        dataset = []
        for line in list(f)[1:]:
            id, text, label = line.strip().split("\t")
            item = {
                "id": id, "text": text, "label": label
            }
            dataset.append(item)
    return dataset

## Prepare Dataset and Labels

In [2]:
DIR = Path("data")

In [3]:
dataset = read_event_dataset(DIR / "test_set_final_release_with_labels.tsv")
texts = [x["text"] for x in dataset]
y_true = [x["label"] for x in dataset]

In [4]:
with open(DIR / "acled_label_to_name.json") as f:
    label_to_text = json.load(f)
    
label_names = sorted(label_to_text)
label_texts = [label_to_text[l] for l in label_names]

In [5]:
ZS_LABELS = ["ORG_CRIME", "NATURAL_DISASTER", "MAN_MADE_DISASTER", "DIPLO", "ATTRIB"]

## Implementing Simple Zero-Shot Classification

In [6]:
def classify_with_cosine(
        model,
        label_names,
        label_texts=None,
        input_texts=None,
        input_embeddings=None,
        label_embeddings=None
    ):
    """
    * label_names: official names/ids of labels that are outputted as predictions
    * label_texts: texts to build label representations from
    """

    if label_embeddings is None:
        label_embeddings = model.encode(label_texts)

    if input_embeddings is None:
        input_embeddings = model.encode(input_texts)
        
    S = util.pytorch_cos_sim(input_embeddings, label_embeddings)    
    predicted_labels = []
    
    for i in range(len(input_embeddings)):
        label_scores = S[i]                
        scored = sorted(zip(label_names, label_scores), key=lambda x: x[1], reverse=True)
        pred = scored[0][0]
        predicted_labels.append(pred)        
    
    return predicted_labels


def evaluate(true_labels, pred_labels, label_set=None):
    for avg in ["micro", "macro", "weighted"]:        
        p, r, f, _ = precision_recall_fscore_support(true_labels, pred_labels, average=avg, labels=label_set)
        gap = " " * (9 - len(avg))
        print(f"{avg}{gap}precision: {p:.3f}, recall: {r:.3f}, f-score: {f:.3f}")

## Predicting and Evaluating Labels

In [7]:
model = SentenceTransformer("paraphrase-mpnet-base-v2", device="cpu")

You try to use a model that was created with version 1.2.0, however, your version is 1.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [8]:
predicted_labels = classify_with_cosine(
    model,
    label_names,
    label_texts=label_texts,
    input_texts=texts,
)

In [9]:
len(predicted_labels)

1019

In [11]:
evaluate(y_true, predicted_labels)

micro    precision: 0.520, recall: 0.520, f-score: 0.520
macro    precision: 0.528, recall: 0.495, f-score: 0.461
weighted precision: 0.569, recall: 0.520, f-score: 0.489


/home/demian/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
evaluate(y_true, predicted_labels, label_set=ZS_LABELS)

micro    precision: 0.782, recall: 0.358, f-score: 0.491
macro    precision: 0.871, recall: 0.383, f-score: 0.467
weighted precision: 0.870, recall: 0.358, f-score: 0.431
